<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/09_IngestaoFreshdesk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install furl # modulo para trabalhar com url
import pandas as pd
import json
import numpy as np
import requests
from pandas.io.json import json_normalize
import time
import datetime
from furl import furl
import re
from google.cloud import bigquery
# Bibliotecas Google Golab
from google.colab import auth
auth.authenticate_user()
# Bibliotecas Google Sheet
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe
# Montar Drive
from google.colab import drive
drive.mount('/content/drive')
# Modulos 
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content
!cp /content/drive/MyDrive/Colab_Notebooks/Autenticacoes/chaves_tokens.py /content
%run chaves_tokens.py
# Importar biblioteca biqquery
from modulos_colab import dados


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


Chave Autenticação API


In [ ]:
# Autenticações Freshdesk
#desk = 'donatelo.freshdesk.com'
#desk_URL = 'https://' + desk
#cat_query = requests.get(desk_URL + "/api/v2/tickets?filter=new_and_my_open", 
#                         auth = auth_deets).json()

#df = pd.DataFrame()
#objects = json_normalize(cat_query)
#df = df.append(objects, ignore_index = True)

# Chave API Freshdesk
auth_deets = (chave_api_freshdesk, 'x')

Ingestão: Grupos, Agentes e Sellers

In [ ]:
# Ingestão BigQuery Grupos, Angentes e Sellers

# Função para extrair dados API - FreshDesk
def data_extraction(endpoint, url):
    """A função chama uma API dependendo do endpoint, salva o resultado que posteriormente é mesclado com as tabelas relevantes"""
    i = 1
    page_num = 300 #the max number of pages allowded
    #initiating empty dataframe
    df = pd.DataFrame()
    while i <= page_num:
        param = { 'page': i, 'per_page': 100}
        formatted_url = furl(url).add(param).url
        #r = requests.get(desk_URL + "/api/v2/tickets?filter=new_and_my_open", auth = auth_deets)
        r = requests.get(formatted_url, auth =auth_deets)
        if r.status_code == 200:
            response = r.json()
            objects = json_normalize(response) #parsing json
            df = df.append(objects, ignore_index = True) #appending the records
            df.fillna(value = '', inplace = True) #replacing NA with empty values
            df.columns
            for c in df.columns:
                df[c] = df[c].astype(str).str.strip('[') #substituindo colchetes por nada
                df[c] = df[c].astype(str).str.strip(']')                
            if endpoint == 'tickets':
                col = ['created_at', 'updated_at', 'due_by'] #alterando o tipo de dados para datas, pois o tipo de dados padrão é objeto 
                for cols in col:
                    df[cols] = pd.to_datetime(df[cols]) 
            else: 
                col = ['created_at', 'updated_at']
                for cols in col:
                    df[cols] = pd.to_datetime(df[cols])                 
            i += 1        
            time.sleep(2)  
        else:
          print("Falha ao ler os tickets, os erros são exibidos abaixo,")
          response = json.loads(r.content)
          print(response["errors"])   
    # Saída dataframe
    return df
        
# Instanciar função        
groups = data_extraction('groups','https://donatelo.freshdesk.com/api/v2/groups')
agents = data_extraction('agents','https://donatelo.freshdesk.com/api/v2/agents')
companies = data_extraction('companies','https://donatelo.freshdesk.com/api/v2/companies')

# Tabela Groups
groups.to_gbq(destination_table='marketplace_analytics.freshdesk_groups',project_id=projeto_gcp_1, if_exists='replace')

# Tabela Agentes
agents.rename(columns={'contact.active':'contact_active',
                       'contact.email':'contact_email',
                       'contact.job_title':'contact_job_title',
                       'contact.language':'contact_language',
                       'contact.mobile':'contact_mobile',
                       'contact.name':'contact_name',
                       'contact.phone':'contact_phone',
                       'contact.time_zone':'contact_time_zone',
                       'contact.created_at':'contact_created_at',
                        'contact.updated_at':'contact_updated_at',
                        'contact.avatar.id':'contact_avatar_id',
                        'contact.avatar.name':'contact_avatar_name',
                        'contact.avatar.content_type':'contact_avatar_content_type',
                        'contact.avatar.size':'contact_avatar_size',
                        'contact.avatar.created_at':'contact_avatar_created_at',
                        'contact.avatar.updated_at':'contact_avatar_updated_at',
                        'contact.avatar.attachment_url':'contact_avatar_attachment_url',
                        'contact.avatar.thumb_url':'contact_avatar_thumb_url',
                        'contact.last_login_at':'contact_last_login_at'}, inplace=True)

agents.to_gbq(destination_table='marketplace_analytics.freshdesk_agentes',project_id=projeto_gcp_1, if_exists='replace')

# Tabela Companies
companies.rename(columns={'custom_fields.erp':'custom_fields_erp',
                          'custom_fields.cnpj':'custom_fields_cnpj'}, inplace=True)
companies.to_gbq(destination_table='marketplace_analytics.freshdesk_companies',project_id=projeto_gcp_1, if_exists='replace')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
1it [00:06,  6.64s/it]
1it [00:06,  6.07s/it]
1it [00:15, 15.34s/it]


Ingestão Dados: Csat, Tickets

In [ ]:
# Função para extrair dados API - FreshDesk
def data_extraction(endpoint, url):
    """A função chama uma API dependendo do endpoint, salva o resultado que posteriormente é mesclado com as tabelas relevantes"""
    i = 1
    page_num = 300 #the max number of pages allowded
    #initiating empty dataframe
    df = pd.DataFrame()
    while i <= page_num:
        param = { 'page': i, 'per_page': 100}
        formatted_url = furl(url).add(param).url
        r = requests.get(formatted_url, auth=auth_deets)
        if r.status_code == 200:
            response = r.json()
            objects = json_normalize(response) #parsing json
            df = df.append(objects, ignore_index = True) #appending the records
            df.fillna(value = '', inplace = True) #replacing NA with empty values
            df.columns
            for c in df.columns:
                df[c] = df[c].astype(str).str.strip('[') #substituindo colchetes por nada
                df[c] = df[c].astype(str).str.strip(']')                
            if endpoint == 'tickets':
                col = ['created_at', 'updated_at', 'due_by'] #alterando o tipo de dados para datas, pois o tipo de dados padrão é objeto 
                for cols in col:
                    df[cols] = pd.to_datetime(df[cols]) 
            else: 
                col = ['created_at', 'updated_at']
                for cols in col:
                    df[cols] = pd.to_datetime(df[cols])                 
            i += 1        
            time.sleep(2)  
        else:
          print("Falha ao ler os tickets, os erros são exibidos abaixo,")
          response = json.loads(r.content)
          print(response["errors"])   
    # Saída dataframe
    return df
        

#tickets = data_extraction('tickets','https://donatelo.freshdesk.com/api/v2/tickets?updated_since=2022-03-31T02:00:00Z')
#tickets = data_extraction('tickets','https://donatelo.freshdesk.com//api/v2/search/tickets?query="status:3"')
#status = data_extraction('tickets','https://donatelo.freshdesk.com/api/v2/search/tickets?query="priority:4%20OR%20priority:3"')
csat = data_extraction('surveys','https://donatelo.freshdesk.com/api/v2/surveys/satisfaction_ratings?created_since=2022-06-01T02:00:00Z')



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [ ]:
# Análise Csat
csat['created_at'] = csat['created_at'].astype(object)
csat['created_at'] = pd.to_datetime(csat['created_at'], format='%Y%m%d')
csat['mes_criacao'] = csat['created_at'].dt.month
csat.rename(columns={'ratings.default_question':'ratings_csat',
                     'ratings.question_22000050441':'ratings_ces'}, inplace=True)

def ajuste_agent_id(csat):
    return csat['agent_id'][0:11]
csat['agent_id'] = csat.apply(ajuste_agent_id, axis=1)

#marco_2022 = csat.query('mes_criacao==3')
#abril_2022 = csat.query('mes_criacao==4')
#maio_2022 = csat.query('mes_criacao==5')
junho_2022 = csat.query('mes_criacao==6')

junho_2022_bigquery = junho_2022[['id','survey_id', 'user_id', 'agent_id', 'feedback','group_id', 'ticket_id', 'created_at', 'updated_at', 'ratings_csat', 'ratings_ces']]
#maio_2022_bigquery = maio_2022[['id','survey_id', 'user_id', 'agent_id', 'feedback','group_id', 'ticket_id', 'created_at', 'updated_at', 'ratings_csat', 'ratings_ces']]
# Ingestão BigQuery
junho_2022_bigquery.to_gbq(destination_table='marketplace_analytics.freshdesk_csat_ces_temp',project_id=projeto_gcp_1, if_exists='replace')
#maio_2022_bigquery.to_gbq(destination_table='marketplace_analytics.freshdesk_csat_ces',project_id= projeto_gcp_1, if_exists='append')



#janeiro_2022 = csat.query('mes_criacao==1')
#fevereiro_2022 = csat.query('mes_criacao==2')
#janeiro_fevereiro_2022 = pd.concat([janeiro_2022,fevereiro_2022])

#def ajuste_agent_id(janeiro_fevereiro_2022):
#    return janeiro_fevereiro_2022['agent_id'][0:11]

#janeiro_fevereiro_2022['agent_id'] = janeiro_fevereiro_2022.apply(ajuste_agent_id, axis=1)
#janeiro_fevereiro_2022_bigQuery = janeiro_fevereiro_2022[['id','survey_id', 'user_id', 'agent_id', 'feedback','group_id', 'ticket_id', 'created_at', 'updated_at', 'ratings_csat', 'ratings_ces']]
#janeiro_fevereiro_2022_bigQuery.to_gbq(destination_table='marketplace_analytics.freshdesk_csat_ces',project_id='maga-bigdata', if_exists='replace')
#janeiro_fevereiro_2022.head()


print('Csat Atualizado!')


1it [00:06,  6.28s/it]
1it [00:03,  3.78s/it]

Csat Atualizado!


In [ ]:
# Select Csat Fresdesk BigQuery

"""
WITH total AS (
    select * from `maga-bigdata.marketplace_analytics.freshdesk_csat_ces`
    union all
    select * from `maga-bigdata.marketplace_analytics.freshdesk_csat_ces_temp`
),
agentes AS (
    select * from `maga-bigdata.marketplace_analytics.freshdesk_agentes`
),
grupos AS (
     select * from `maga-bigdata.marketplace_analytics.freshdesk_groups`
)
SELECT
    cast(DATETIME_ADD(CAST(total.created_at AS DATETIME), INTERVAL -3 HOUR) as date) as created_at, 
    cast(DATETIME_ADD(CAST(total.updated_at AS DATETIME), INTERVAL -3 HOUR) as date) as updated_at, 
    --total.created_at,
    --total.updated_at,
    total.ratings_csat,
    total.ratings_ces,
    total.ticket_id,
    total.feedback,
    agentes.contact_name,
    agentes.contact_email,
    grupos.name as grupo,
    case
        when total.ratings_csat = '103' then 'Extremamente Feliz'
        when total.ratings_csat = '102' then 'Feliz'
        when total.ratings_csat = '101' then 'Feliz'
        when total.ratings_csat = '100' then 'Neutro'
        when total.ratings_csat = '-101' then 'Infeliz'
        when total.ratings_csat = '-102' then 'Infeliz'
        when total.ratings_csat = '-103' then 'Extremanente Infeliz'
        else ('nulo')
        end as nota_csat,
    case
        when total.ratings_ces = '103.0' then 'Muito Fácil'
        when total.ratings_ces = '102.0' then 'Fácil'
        when total.ratings_ces = '101.0' then 'Fácil'
        when total.ratings_ces = '100.0' then 'Nem fácil, nem dificil'
        when total.ratings_ces = '-101.0' then 'dificil'
        when total.ratings_ces = '-102.0' then 'dificil'
        when total.ratings_ces = '-103.0' then 'Muito dificil'
        else ('nulo')
        end as nota_ces
FROM total
    left join agentes on agentes.id = total.agent_id
    left join grupos on grupos.id = total.group_id
    
"""